In [4]:
!pip install sagemaker==1.0.0

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\sagar\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\sagar\\AppData\\Local\\Temp\\pip-install-blld7fxl\\sagemaker\\setup.py'"'"'; __file__='"'"'C:\\Users\\sagar\\AppData\\Local\\Temp\\pip-install-blld7fxl\\sagemaker\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\sagar\AppData\Local\Temp\pip-install-blld7fxl\sagemaker\pip-egg-info'
         cwd: C:\Users\sagar\AppData\Local\Temp\pip-install-blld7fxl\sagemaker\
    Complete output (9 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\sagar\AppData\Local\Temp\pip-install-blld7fxl\sagemaker\setup.py", line 18, in <module>
        long_description=read('README.rst'),
      File "C:\Users\sagar\AppData\Local\Te

In [1]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import time
import numpy as np
import pandas as pd
s3 = boto3.resource('s3')

sagemaker_session = sagemaker.Session()
role = get_execution_role()

BUCKET      = sagemaker_session.default_bucket()
TRAINING_FILE     = 'prediction2.py'
INFERENCE_FILE = 'inference.py'
SOURCE_DIR = 'source_dir'

DATA_PREFIX            = 'RF_SCIKIT'
MULTI_MODEL_ARTIFACTS  = 'multi_model_artifacts'

TRAIN_INSTANCE_TYPE    = 'ml.m4.xlarge'
ENDPOINT_INSTANCE_TYPE = 'ml.m4.xlarge'

ENDPOINT_NAME = 'model-random-forest-new'

MODEL_NAME = ENDPOINT_NAME

In [2]:
from sklearn.model_selection import train_test_split

SEED = 7
SPLIT_RATIOS = [0.6, 0.3, 0.1]
import boto3
def split():
    bucket_name = 'sagemaker-data-icr'
    s3 = boto3.resource('s3')
    data_key = 'first phenomes and genomes.csv'
    data_location = 's3://{}/{}'.format(bucket_name, data_key)
    df = pd.read_csv(data_location)
    #data = df.drop(['Unnamed: 0'],axis=1)
    model_data = df.fillna(df.mean())
    COLUMNS = list(df.columns)
    
#     #i = str(input("enter a feature name "))
#     X =df1.drop([loc], axis = 1)
   
#     y =df1[loc]
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
#     #X_train_col = X_train.columns
#     return X_train, X_test, y_train, y_test
    
    #i = str(input("enter a feature name "))
    #for i in col:
   
    # split data into train and test sets
    seed      = SEED
    val_size  = SPLIT_RATIOS[1]
    test_size = SPLIT_RATIOS[2]
    
    _train, _val, _test = np.split(model_data.sample(frac=1, random_state=123), 
                                                  [int(0.7 * len(model_data)), int(0.9*len(model_data))]) 
    #train =  train_data.drop([i], axis=1)#, index=False)
    #val =   validation_data.drop([i], axis=1)#, index=False)
    #test =  test_data.drop([i], axis=1)#, index=False)

# Dropping the target value, as we will use this CSV file for batch transform
    #test = test_data.drop([i], axis=1).to_csv('test.csv', index=False, header=False)

    return _train, _val, _test ,COLUMNS


In [3]:
import os
def save_data_locally(location, train, val, test,COLUMNS):

#     header = ','.join(COLUMNS)
#     print(header)
#     print(train)
    os.makedirs(f'data/{location}/train')
    np.savetxt(f'data/{location}/train/{location}_train.csv', train, header=','.join(COLUMNS), delimiter=',', fmt='%.10f',comments="")
    
    os.makedirs(f'data/{location}/val')
    np.savetxt(f'data/{location}/val/{location}_val.csv',     val,  header = ','.join(COLUMNS), delimiter=',', fmt='%.10f',comments="")
    
    os.makedirs(f'data/{location}/test')
    np.savetxt(f'data/{location}/test/{location}_test.csv',   test,  header = ','.join(COLUMNS),  delimiter=',', fmt='%.10f',comments="")


In [17]:
from sagemaker.sklearn.estimator import SKLearn

def launch_training_job(location):
    # clear out old versions of the data
    s3_bucket = s3.Bucket(BUCKET)
    full_input_prefix = f'{DATA_PREFIX}/model_prep/{location}'
    s3_bucket.objects.filter(Prefix=full_input_prefix + '/').delete()

    # upload the entire set of data for all three channels
    local_folder = f'data/{location}'
    inputs = sagemaker_session.upload_data(path=local_folder, 
                                            key_prefix=full_input_prefix)
    
    print(f'Training data uploaded: {inputs}')
    
    _job = 'gp-{}'.format(location.replace('_', '-'))
    full_output_prefix = f'{DATA_PREFIX}/model_artifacts/{location}'
    s3_output_path = f's3://{BUCKET}/{full_output_prefix}'
    
    code_location = f's3://{BUCKET}/{full_input_prefix}/code'
    # Return the estimator object
    return s3_output_path,_job, code_location ,inputs,location 

from sagemaker.sklearn.estimator import SKLearn
_train, _val, _test ,COLUMNS  = split()

LOCATIONS  = COLUMNS
PARALLEL_TRAINING_JOBS =2

import shutil
import os

estimators = []
_train, _val, _test ,COLUMNS  = split()
shutil.rmtree('data', ignore_errors=True)
for loc in LOCATIONS[:PARALLEL_TRAINING_JOBS]:
    _train, _val, _test ,COLUMNS = split()
    save_data_locally(loc, _train, _val, _test ,COLUMNS)
    
    s3_output_path,_job, code_location,inputs,location = launch_training_job(loc)
    _estimator = SKLearn(
        entry_point=TRAINING_FILE, # script to use for training job
        role=role,
        source_dir=SOURCE_DIR, # Location of scripts
        train_instance_count=1,
        train_instance_type=TRAIN_INSTANCE_TYPE,
        framework_version='0.23-1',# 0.23-1 is the latest version
        output_path=s3_output_path,# Where to store model artifacts
        base_job_name=_job,
        code_location=code_location,
        hyperparameters =  {'model-name': location})
    
    
    DISTRIBUTION_MODE = 'FullyReplicated'
    
    
    train_input = sagemaker.s3_input(s3_data=inputs+'/train', 
                                      distribution=DISTRIBUTION_MODE, content_type='csv')
    
    val_input   = sagemaker.s3_input(s3_data=inputs+'/val', 
                                      distribution=DISTRIBUTION_MODE, content_type='csv')
    
    remote_inputs = {'train': train_input, 'validation': val_input}
  
    
    #eval_set=[(train_input2, val_input2)]
    #eval_set = {'train': train_input2,'test': val_input2}


    _estimator.fit(remote_inputs,wait = False)#,eval_set)
    estimators.append(_estimator)

    
    print('{} training jobs launched: {}'.format(len(estimators), estimators))
    

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Training data uploaded: s3://sagemaker-ap-south-1-580246529711/RF_SCIKIT/model_prep/100SDW_EIAR_2008_RF
1 training jobs launched: [<sagemaker.sklearn.estimator.SKLearn object at 0x7fb39e83f940>]


's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Training data uploaded: s3://sagemaker-ap-south-1-580246529711/RF_SCIKIT/model_prep/100SDW_EIAR_2009_RF
2 training jobs launched: [<sagemaker.sklearn.estimator.SKLearn object at 0x7fb39e83f940>, <sagemaker.sklearn.estimator.SKLearn object at 0x7fb39f5a4e80>]


In [18]:
#APB_PAT_2001_RF2
#[100SDW_EIAR_2009_RF','100SDW_EU_2008_RF']

#2014DSPHY-1nd-layer
print()
print(f'{len(estimators)} training jobs launched: {[x.latest_training_job.name for x in estimators]}')


2 training jobs launched: ['gp-100SDW-EIAR-2008-RF-2020-07-28-15-09-06-203', 'gp-100SDW-EIAR-2009-RF-2020-07-28-15-09-13-556']


In [19]:
def wait_for_training_job_to_complete(estimator):
    job = estimator.latest_training_job.job_name
    print(f'Waiting for job: {job}')
    status = estimator.latest_training_job.describe()['TrainingJobStatus']
    while status == 'InProgress':
        time.sleep(45)
        status = estimator.latest_training_job.describe()['TrainingJobStatus']
        if status == 'InProgress':
            print(f'{job} job status: {status}')
    print(f'DONE. Status for {job} is {status}\n')
# wait for the jobs to finish
for est in estimators:
    wait_for_training_job_to_complete(est)


Waiting for job: gp-100SDW-EIAR-2008-RF-2020-07-28-15-09-06-203
gp-100SDW-EIAR-2008-RF-2020-07-28-15-09-06-203 job status: InProgress
gp-100SDW-EIAR-2008-RF-2020-07-28-15-09-06-203 job status: InProgress
gp-100SDW-EIAR-2008-RF-2020-07-28-15-09-06-203 job status: InProgress
gp-100SDW-EIAR-2008-RF-2020-07-28-15-09-06-203 job status: InProgress
gp-100SDW-EIAR-2008-RF-2020-07-28-15-09-06-203 job status: InProgress
gp-100SDW-EIAR-2008-RF-2020-07-28-15-09-06-203 job status: InProgress
gp-100SDW-EIAR-2008-RF-2020-07-28-15-09-06-203 job status: InProgress
DONE. Status for gp-100SDW-EIAR-2008-RF-2020-07-28-15-09-06-203 is Failed

Waiting for job: gp-100SDW-EIAR-2009-RF-2020-07-28-15-09-13-556
DONE. Status for gp-100SDW-EIAR-2009-RF-2020-07-28-15-09-13-556 is Failed



In [3]:
from sagemaker.sklearn.estimator import SKLearn
job_name='gp-100SDW-EIAR-2008-RF-2020-07-28-15-09-06-203'
my_estimator = SKLearn.attach(job_name)
my_estimator

#aws sagemaker describe-training-job --training-job-name 'GENOME-PHENOME-2020-05-27-13-03-03'

ModuleNotFoundError: No module named 'sagemaker'

In [ ]:
estimator = estimators[0]
# inference.py is the entry_point for when we deploy the model
# Note how we do NOT specify source_dir again, this information is inherited from the estimator
model = estimator.create_model(role=role, entry_point='inference.py')

In [ ]:
from sagemaker.multidatamodel import MultiDataModel

# This is where our MME will read models from on S3.
model_data_prefix = f's3://{BUCKET}/{DATA_PREFIX}/{MULTI_MODEL_ARTIFACTS}/'

gp = MultiDataModel(name=MODEL_NAME,
                     model_data_prefix=model_data_prefix,
                     model=model,# passing our model
                     sagemaker_session=sagemaker_session)

In [ ]:
predictor = gp.deploy(initial_instance_count=1,
                       instance_type=ENDPOINT_INSTANCE_TYPE,
                       endpoint_name=ENDPOINT_NAME)

In [ ]:
# No models visible!
list(gp.list_models())

In [ ]:
for est in estimators:
    artifact_path = est.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
    model_name = artifact_path.split('/')[-4]+'.tar.gz'
    # This is copying over the model artifact to the S3 location for the MME.
    gp.add_model(model_data_source=artifact_path, model_data_path=model_name)

In [ ]:
list(gp.list_models())


In [ ]:

start_time = time.time()
out_arr = np.random.randint(low = 0, high = 1099, size = 4799)

predicted_value = predictor.predict(out_arr, target_model='100SDW_EIAR_2009_RF.tar.gz')

duration = time.time() - start_time
print('${:,.2f}, took {:,d} ms\n'.format(predicted_value[0], int(duration * 1000)))

In [ ]:
from sagemaker.sklearn.estimator import SKLearn
def launch_training_job(i):
    

    inputs = sagemaker_session.upload_data('data')

    print(f'Training data uploaded: {inputs}')

    _job = 'gp-{}'.format(i.replace('_', '-'))
    inputs = sagemaker_session.upload_data('data')
    #print('Training data uploaded: {}'.format(inputs))
    
    _job = 'gp-{}'.format(i.replace('_', '-'))
    _full_output_prefix = '{}/model_artifacts/{}'.format(DATA_PREFIX, 
                                                        i)
    _s3_output_path = 's3://{}/{}'.format(BUCKET, _full_output_prefix)
    return _s3_output_path,_job

    #return s3_output_path ,code_location ,_job


    


In [ ]:

from sagemaker.sklearn.estimator import SKLearn

LOCATIONS  = ['100SDW_EIAR_2008_RF','100SDW_EIAR_2009_RF','100SDW_EU_2008_RF','100SDW_EU_2009_RF']
PARALLEL_TRAINING_JOBS = 2
estimators = []
for loc in LOCATIONS[:PARALLEL_TRAINING_JOBS]:
    train_data, validation_data, test_data = split() 
    #df1= train_validate_test_split(df1)
    #save_data_locally(loc,df1)
   # _s3_output_path,_job = launch_training_job(loc)
    # clear out old versions of the data
    
    _s3_output_path,_job = launch_training_job(i)
    _estimator = SKLearn(
         entry_point='train_prediction.py', role=role,
         train_instance_count=1, train_instance_type=TRAIN_INSTANCE_TYPE,
         framework_version='0.20.0',
         output_path=_s3_output_path,
         base_job_name=_job)
    
    DISTRIBUTION_MODE = 'FullyReplicated'
    
    train_input = sagemaker_session.upload_data("data")
    _remote_inputs = {'train': train_input}
    _estimator.fit(_remote_inputs, wait=False)

    #_remote_inputs = {'train': train_input}
    #_estimator.fit(_remote_inputs, wait=False)
    estimators.append( _estimator)
    print('{} training jobs launched: {}'.format(len(estimators), estimators))

In [ ]:
#APB_PAT_2001_RF2
#[100SDW_EIAR_2009_RF','100SDW_EU_2008_RF']

#2014DSPHY-1nd-layer
print()
print(f'{len(estimators)} training jobs launched: {[x.latest_training_job.name for x in estimators]}')

In [ ]:
def wait_for_training_job_to_complete(estimator):
    job = estimator.latest_training_job.job_name
    print(f'Waiting for job: {job}')
    status = estimator.latest_training_job.describe()['TrainingJobStatus']
    while status == 'InProgress':
        time.sleep(45)
        status = estimator.latest_training_job.describe()['TrainingJobStatus']
        if status == 'InProgress':
            print(f'{job} job status: {status}')
    print(f'DONE. Status for {job} is {status}\n')
# wait for the jobs to finish
for est in estimators:
    wait_for_training_job_to_complete(est)



In [ ]:
estimator = estimators[0]
# inference.py is the entry_point for when we deploy the model
# Note how we do NOT specify source_dir again, this information is inherited from the estimator
model = estimator.create_model(role=role)#, entry_point='inference.py')

In [ ]:
model

In [ ]:
from sagemaker.multidatamodel import MultiDataModel

# This is where our MME will read models from on S3.
model_data_prefix = f's3://{BUCKET}/{DATA_PREFIX}/{MULTI_MODEL_ARTIFACTS}/'

mme = MultiDataModel(name=MODEL_NAME,
                     model_data_prefix=model_data_prefix,
                     model=model,# passing our model
                     sagemaker_session=sagemaker_session)

In [ ]:

predictor = mme.deploy(initial_instance_count=1,
                       instance_type=ENDPOINT_INSTANCE_TYPE,
                       endpoint_name='gp-sklearn')

In [ ]:
import numpy as np
import pandas as pd
import time

NUM_HOUSES_PER_LOCATION = 1000
LOCATIONS  = ['NewYork_NY',    'LosAngeles_CA',   'Chicago_IL',    'Houston_TX',   'Dallas_TX',
              'Phoenix_AZ',    'Philadelphia_PA', 'SanAntonio_TX', 'SanDiego_CA',  'SanFrancisco_CA']
PARALLEL_TRAINING_JOBS = 2 # len(LOCATIONS) if your account limits can handle it
MAX_YEAR = 2019

def gen_price(house):
    _base_price = int(house['SQUARE_FEET'] * 150)
    _price = int(_base_price + (10000 * house['NUM_BEDROOMS']) + \
                               (15000 * house['NUM_BATHROOMS']) + \
                               (15000 * house['LOT_ACRES']) + \
                               (15000 * house['GARAGE_SPACES']) - \
                               (5000 * (MAX_YEAR - house['YEAR_BUILT'])))
    return _price

def gen_random_house():
    _house = {'SQUARE_FEET':   int(np.random.normal(3000, 750)),
              'NUM_BEDROOMS':  np.random.randint(2, 7),
              'NUM_BATHROOMS': np.random.randint(2, 7) / 2,
              'LOT_ACRES':     round(np.random.normal(1.0, 0.25), 2),
              'GARAGE_SPACES': np.random.randint(0, 4),
              'YEAR_BUILT':    min(MAX_YEAR, int(np.random.normal(1995, 10)))}
    _price = gen_price(_house)
    return [_price, _house['YEAR_BUILT'],   _house['SQUARE_FEET'], 
                    _house['NUM_BEDROOMS'], _house['NUM_BATHROOMS'], 
                    _house['LOT_ACRES'],    _house['GARAGE_SPACES']]

def gen_houses(num_houses):
    _house_list = []
    for i in range(num_houses):
        _house_list.append(gen_random_house())
    _df = pd.DataFrame(_house_list, 
                       columns=['PRICE', 'YEAR_BUILT',
                                'SQUARE_FEET', 'NUM_BEDROOMS',
                                'NUM_BATHROOMS', 'LOT_ACRES',
                                'GARAGE_SPACES'])
    return _df
import sagemaker
from sagemaker import get_execution_role
import boto3

s3 = boto3.resource('s3')

sagemaker_session = sagemaker.Session()
role = get_execution_role()

BUCKET      = sagemaker_session.default_bucket()
TRAINING_FILE     = 'training.py'
INFERENCE_FILE = 'inference.py'
SOURCE_DIR = 'source_dir'

DATA_PREFIX            = 'DEMO_MME_SCIKIT_V1'
MULTI_MODEL_ARTIFACTS  = 'multi_model_artifacts'

TRAIN_INSTANCE_TYPE    = 'ml.m4.xlarge'
ENDPOINT_INSTANCE_TYPE = 'ml.m4.xlarge'

ENDPOINT_NAME = 'housing-V1'

MODEL_NAME = ENDPOINT_NAME

from sklearn.model_selection import train_test_split
SEED = 7
SPLIT_RATIOS = [0.6, 0.3, 0.1]

def split_data(df):
    # split data into train and test sets
    seed      = SEED
    val_size  = SPLIT_RATIOS[1]
    test_size = SPLIT_RATIOS[2]
    
    num_samples = df.shape[0]
    X1 = df.values[:num_samples, 1:] # keep only the features, skip the target, all rows
    Y1 = df.values[:num_samples, :1] # keep only the target, all rows

    # Use split ratios to divide up into train/val/test
    X_train, X_val, y_train, y_val = \
        train_test_split(X1, Y1, test_size=(test_size + val_size), random_state=seed)
    # Of the remaining non-training samples, give proper ratio to validation and to test
    X_test, X_test, y_test, y_test = \
        train_test_split(X_val, y_val, test_size=(test_size / (test_size + val_size)), 
                         random_state=seed)
    # reassemble the datasets with target in first column and features after that
    _train = np.concatenate([y_train, X_train], axis=1)
    _val   = np.concatenate([y_val,   X_val],   axis=1)
    _test  = np.concatenate([y_test,  X_test],  axis=1)

    return _train, _val, _test

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

def launch_training_job(location):
    # clear out old versions of the data
    s3_bucket = s3.Bucket(BUCKET)
    full_input_prefix = f'{DATA_PREFIX}/model_prep/{location}'
    s3_bucket.objects.filter(Prefix=full_input_prefix + '/').delete()

    # upload the entire set of data for all three channels
    local_folder = f'data/{location}'
    inputs = sagemaker_session.upload_data(path=local_folder, 
                                            key_prefix=full_input_prefix)
    
    print(f'Training data uploaded: {inputs}')
    
    _job = 'skl-{}'.format(location.replace('_', '-'))
    full_output_prefix = f'{DATA_PREFIX}/model_artifacts/{location}'
    s3_output_path = f's3://{BUCKET}/{full_output_prefix}'
    
    code_location = f's3://{BUCKET}/{full_input_prefix}/code'
    

    # Add code_location argument in order to ensure that code_artifacts are stored in the same place.
    estimator = SKLearn(
        entry_point=TRAINING_FILE, # script to use for training job
        role=role,
        source_dir=SOURCE_DIR, # Location of scripts
        train_instance_count=1,
        train_instance_type=TRAIN_INSTANCE_TYPE,
        framework_version='0.23-1',# 0.23-1 is the latest version
        output_path=s3_output_path,# Where to store model artifacts
        base_job_name=_job,
        code_location=code_location,# This is where the .tar.gz of the source_dir will be stored
        metric_definitions=[
            {'Name' : 'median-AE',
             'Regex': 'AE-at-50th-percentile: ([0-9.]+).*$'}],
        hyperparameters = {'n-estimators'    : 100,
                            'min-samples-leaf': 3,
                            'model-name'      : location})
    
    DISTRIBUTION_MODE = 'FullyReplicated'
    
    train_input = sagemaker.s3_input(s3_data=inputs+'/train', 
                                      distribution=DISTRIBUTION_MODE, content_type='csv')
    
    val_input   = sagemaker.s3_input(s3_data=inputs+'/val', 
                                      distribution=DISTRIBUTION_MODE, content_type='csv')
    
    remote_inputs = {'train': train_input, 'validation': val_input}

    estimator.fit(remote_inputs, wait=False)
    
    # Return the estimator object
    return estimator

def save_data_locally(location, train, val, test):
#     _header = ','.join(COLUMNS)
    
    os.makedirs(f'data/{location}/train')
    np.savetxt(f'data/{location}/train/{location}_train.csv', train, delimiter=',', fmt='%.2f')
    
    os.makedirs(f'data/{location}/val')
    np.savetxt(f'data/{location}/val/{location}_val.csv',     val,   delimiter=',', fmt='%.2f')
    
    os.makedirs(f'data/{location}/test')
    np.savetxt(f'data/{location}/test/{location}_test.csv',   test,  delimiter=',', fmt='%.2f')

import shutil
import os

estimators = []

shutil.rmtree('data', ignore_errors=True)

for loc in LOCATIONS[:PARALLEL_TRAINING_JOBS]:
    _houses = gen_houses(NUM_HOUSES_PER_LOCATION)
    _train, _val, _test = split_data(_houses)
    save_data_locally(loc, _train, _val, _test)
    estimator = launch_training_job(loc)
    estimators.append(estimator)
    time.sleep(2) # to avoid throttling the CreateTrainingJob API

print()
print(f'{len(estimators)} training jobs launched: {[x.latest_training_job.job_name for x in estimators]}')

def wait_for_training_job_to_complete(estimator):
    job = estimator.latest_training_job.job_name
    print(f'Waiting for job: {job}')
    status = estimator.latest_training_job.describe()['TrainingJobStatus']
    while status == 'InProgress':
        time.sleep(45)
        status = estimator.latest_training_job.describe()['TrainingJobStatus']
        if status == 'InProgress':
            print(f'{job} job status: {status}')
    print(f'DONE. Status for {job} is {status}\n')

# wait for the jobs to finish
for est in estimators:
    wait_for_training_job_to_complete(est)


In [ ]:
from sagemaker.sklearn.estimator import SKLearn
job_name="gp-100SDW-EIAR-2008-RF-2020-07-21-08-41-59-466"
my_estimator = SKLearn.attach(job_name)
my_estimator

#aws sagemaker describe-training-job --training-job-name 'GENOME-PHENOME-2020-05-27-13-03-03'

In [ ]:
train.shape[0]

In [ ]:
#[100SDW_EIAR_2009_RF','100SDW_EU_2008_RF']


_houses = gen_houses(1000)

num_samples = _houses.shape[0]

num_samples
#_train, _val, _test = split_data(_houses)

In [ ]:
import pandas as pd
def gen_hous():
    bucket_name = 'sagemaker-data-icr'
    s3 = boto3.resource('s3')
    data_key = 'first phenomes and genomes.csv'
    data_location = 's3://{}/{}'.format(bucket_name, data_key)
    df = pd.read_csv(data_location)
    #data = df.drop(['Unnamed: 0'],axis=1)
    df1 = df.fillna(df.mean())
    COLUMNS = list(df1.columns)
    
    i = str(input("enter a feature name "))
    #for i in col:
    return i,df1